<center><h1>Text Preprocessing using text mining</h1></center>

<h3><u>Tasks :</u></h3> 
<h4>- Text Cleaning</h4> 
<h4>- Text similarity evaluation</h4> 
<h4>- Extraction of common and uncommon parts</h4> 
<h4>- N gram distribution</h4>  
<h4>- Apply the mask on the uncommon parts</h4> 

## 0) Import the libraries

In [59]:
import numpy as np
import re
import nltk
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder
from nltk.util import ngrams
import spacy

## 1) Text cleaning

In [90]:
# init the sentences to preprocess

sentence1 = "I love to pay my video games in my free time, especially retro video games."
sentence2 = "I love to play oreo games in my free thyme, especially retro video games."
sentence3 = "I live to slay video vames in my free time, especially utro video games."
sentences = np.array([[sentence1], [sentence2], [sentence3]])
# sentence1 = "hello my name is james and I work as a data scientists"
# sentence2 = "hello my nail fits jane andy workable scientist"
# sentences = np.array([[sentence1], [sentence2]])

In [91]:
# Remove punctuation
def preprocess(sentence):
    sentence = sentence[0]
    sentence = sentence.lower()
    sentence = re.sub(r'[^\w\s]', '', sentence)
    return [sentence]
sentences = np.apply_along_axis(preprocess, 1, sentences)
sentences

array([['i love to pay my video games in my free time especially retro video games'],
       ['i love to play oreo games in my free thyme especially retro video games'],
       ['i live to slay video vames in my free time especially utro video games']],
      dtype='<U73')

## 2) Text similarity analysis

In [62]:
# Low priority task will be done using cosine similarity for a starter

## 3) Extraction of common and uncommon parts

### Using Dynamic programming

In [108]:
# Transform the sentences into vectors
def split_preprocess(sentences):
    s = []
    for sentence in sentences:
        s.append(sentence[0].split())
    return s

# Reduce the sequences of # into one #
def shrink(sentence):
    temp = sentence.split()
    b = False
    for i in range(len(temp)):
        if temp[i] == "#" and b:
            temp[i] = ""
        elif temp[i] == "#" and not b:
            b = True
        elif temp[i] != "#" and b:
            b = False
    while "" in temp:       
        temp.remove("")
    
    return " ".join(temp)

# Init the Dynamic matrix
def init_matrix(temp_sentence, sentences, lenght, l):
        # initialize the L matrix with zeros
        L = [[0] * (lenght + 1) for _ in range(len(temp_sentence) + 1)]

        # fill in the L matrix using dynamic programming
        for i in range(len(temp_sentence) + 1):
            for j in range(lenght + 1):
                # if either string is empty, the longest common substring is zero
                if i == 0 or j == 0:
                    L[i][j] = 0
                # if the characters match, add one to the length of the longest common substring
                elif temp_sentence[i - 1] == sentences[l][j - 1]:
                    L[i][j] = L[i - 1][j - 1] + 1
                # if the characters don't match, take the maximum length from the previous row or column
                else:
                    L[i][j] = max(L[i - 1][j], L[i][j - 1])
        return L

# init list of lists
def init_list_of_lists(lenght):
    list_of_lists = []
    for i in range(lenght):
        list_of_lists.append([])
    return list_of_lists

# remove all the occourences of a value in a list
def remove_all(liste, value):
    while value in liste:
        liste.remove(value)
    return liste

In [106]:
def common_and_uncommon_extraction(sentences):
    lens = [len(s) for s in sentences]

    # initialize the uncommon substring lists
    uncommon_str_i = init_list_of_lists(len(sentences))

    temp_sentence = sentences[0]
    for l in range(1, len(sentences)):
        # initialize the L matrix
        L = init_matrix(temp_sentence, sentences, lens[l], l)

        # calculate the index based on the length of the longer string
        index = max(len(temp_sentence), lens[l])

        # initialize the common list with empty strings
        common = [""] * (index + 1)
        common[index] = ""

        # set i and j to the end of each string
        i = len(temp_sentence)
        j = lens[l]

        tracker_str1 = -1
        tracker_str2 = -1
        sub_uncommon_str = []
        sub_uncommon = []
        sub_uncommon_str_i_temp = []
        sub_uncommon_str_temp = init_list_of_lists(len(sentences))

        # loop through the L matrix to find the common and uncommon substrings
        while i > 0 and j > 0:
            # if the characters match, add the character to the common list and move to the previous diagonal cell
            if temp_sentence[i - 1] == sentences[l][j - 1]:
                common[index - 1] = temp_sentence[i - 1]
                i -= 1
                j -= 1
                index -= 1
            # if the length of the substring from the previous column is greater, add the uncommon character to uncommon_str1 list and move to the previous column
            elif L[i - 1][j] < L[i][j - 1]:
                if tracker_str1 == -1: 
                    tracker_str1 = j - 1
                    sub_uncommon_str.append(sentences[l][j - 1])
                elif tracker_str1 == j:
                    sub_uncommon_str.append(sentences[l][j - 1])
                    tracker_str1 = j - 1
                else:
                    sub_uncommon_str.reverse()
                    uncommon_str_i[l].append(sub_uncommon_str if len(sub_uncommon_str) > 1 else sub_uncommon_str[0])
                    sub_uncommon_str = []
                    tracker_str1 = j - 1
                    sub_uncommon_str.append(sentences[l][j - 1])

                j -= 1
            # if the length of the substring from the previous row is greater, add the uncommon character to uncommon_str2 list and move to the previous row
            else:
                if tracker_str2 == -1:
                    tracker_str2 = i - 1
                    sub_uncommon.append(temp_sentence[i - 1])
                elif tracker_str2 == i:
                    sub_uncommon.append(temp_sentence[i - 1])
                    tracker_str2 = i - 1
                else:
                    sub_uncommon.reverse()
                    if l == 1:
                        uncommon_str_i[0].append(sub_uncommon if len(sub_uncommon) > 1 else sub_uncommon[0])
                    else:
                        if '#' not in sub_uncommon:
                            sub_uncommon.reverse()
                            for k in range(l):
                                sub_uncommon_str_temp[k].append(sub_uncommon if len(sub_uncommon) > 1 else sub_uncommon[0])
                        else:
                            sub_uncommon_copy = sub_uncommon.copy()
                            sub_uncommon_str_i_temp.append(sub_uncommon_copy if len(sub_uncommon_copy) > 1 else sub_uncommon_copy[0])
                            for k in range(l):
                                sub_uncommon_copy = sub_uncommon.copy()
                                updated_uncommon_str = uncommon_str_i[k][len(sub_uncommon_str_i_temp) - 1]
                                if type(updated_uncommon_str) == list:
                                    for term in updated_uncommon_str:
                                        if '#' in sub_uncommon_copy:
                                            ind = sub_uncommon_copy.index("#")
                                            sub_uncommon_copy[ind] = term
                                    sub_uncommon_copy = remove_all(sub_uncommon_copy, '#')
                                    sub_uncommon_str_temp[k].append(sub_uncommon_copy if len(sub_uncommon) > 1 else sub_uncommon_copy[0])
                                else:
                                    sub_uncommon_str_temp[k].append(updated_uncommon_str)
                    sub_uncommon = []
                    tracker_str2 = i - 1
                    sub_uncommon.append(temp_sentence[i - 1])

                common[index - 1] = "#"
                index -= 1
                i  -= 1

        if l == 1:
            if len(sub_uncommon) > 0:
                sub_uncommon.reverse()
                uncommon_str_i[0].append(sub_uncommon if len(sub_uncommon) > 1 else sub_uncommon[0])
        else:
            for k in range(l):
                uncommon_str_i[k] = sub_uncommon_str_temp[k]
            if len(sub_uncommon) > 0:
                if '#' not in sub_uncommon:
                    sub_uncommon.reverse()
                    for k in range(l):
                        uncommon_str_i[k].append(sub_uncommon if len(sub_uncommon) > 1 else sub_uncommon[0])
                else:
                    sub_uncommon.reverse()
                    for k in range(l):
                        sub_uncommon_copy = sub_uncommon.copy()
                        for term in uncommon_str_i[k][len(uncommon_str_i[k]) - 1]:
                            if '#' in sub_uncommon_copy:
                                ind = sub_uncommon_copy.index("#")
                                sub_uncommon_copy[ind] = term
                        sub_uncommon_copy = remove_all(sub_uncommon_copy, "#")
                        uncommon_str_i[k][len(uncommon_str_i[k]) - 1] = sub_uncommon_copy

        if len(sub_uncommon_str) > 0:
            sub_uncommon_str.reverse()
            uncommon_str_i[l].append(sub_uncommon_str if len(sub_uncommon_str) > 1 else sub_uncommon_str[0])

        temp_sentence = remove_all(common.copy(), "")

    # join the common list into a sentence
    common_sentence = " ".join(temp_sentence)

    # reverse the order of the uncommon substring lists
    for i in range(len(uncommon_str_i)):
        uncommon_str_i[i].reverse()

    # return the common sentence and the lists of uncommon substrings
    return shrink(common_sentence), uncommon_str_i

In [109]:
for i in range(len(sentences)):
    print("sentence " + str(i + 1) +" : "+ sentences[i][0])

splited_sentences = split_preprocess(sentences)
common_sentence, uncommon_str_i = common_and_uncommon_extraction(splited_sentences)
print("\nCommon parts of the sentences : " + common_sentence)

print("\nUncommon parts of each sentences :")
for i in range(len(uncommon_str_i)):
    print("\nsentence " + str(i + 1) +" : "+str(uncommon_str_i[i]))

sentence 1 : i love to pay my video games in my free time especially retro video games
sentence 2 : i love to play oreo games in my free thyme especially retro video games
sentence 3 : i live to slay video vames in my free time especially utro video games

Common parts of the sentences : i # to # in my free # especially # video games

Uncommon parts of each sentences :

sentence 1 : ['love', ['pay', 'my', 'video', 'games'], 'time', 'retro']

sentence 2 : ['love', ['play', 'oreo', 'games'], 'thyme', 'retro']

sentence 3 : ['live', ['slay', 'video', 'vames'], 'time', 'utro']


## 4) N gram distribution

In [66]:
def get_gram_lentgh(uncommon_str_i):
    lens = []
    for i in range(len(uncommon_str_i[0])):
        temp = []
        for j in range(len(uncommon_str_i)):
            temp.append(len(uncommon_str_i[j][i]) if type(uncommon_str_i[j][i]) == list else 1)
        lens.append(min(temp))
    return lens

def get_og_sentence_vector(uncommon_str, common_sentence):
    og_sentence_vector = []
    temp = common_sentence.split()
    i = 0    
    for t in temp:
        if t == "#":
            if type(uncommon_str[i]) == list:
                og_sentence_vector.extend(uncommon_str[i])
            else:
                og_sentence_vector.append(uncommon_str[i])
            i += 1
        else:
            og_sentence_vector.append(t)
    return og_sentence_vector

In [67]:
def ngram_distribution(uncommon_str_i, common_sentence):
    final_uncommon_str_i = []
    for i in range(len(uncommon_str_i)):
        final_uncommon_str_i.append([])

    nb_unc_str = 0

    lens = get_gram_lentgh(uncommon_str_i)

    for uncommon_str in uncommon_str_i:
        for i in range(len(uncommon_str)):
            # Make a copy of the current list of the current uncommon part for string 1
            unc_str = uncommon_str[i].copy() if type(uncommon_str[i]) == list else [uncommon_str[i]]
            while len(unc_str) > lens[i]:
                og_sentence = get_og_sentence_vector(uncommon_str, common_sentence)
                bigram_measures = BigramAssocMeasures()

                # Variable containing the common words that won't allowed in the bigrams
                common_words_str = list(set(og_sentence) - set(unc_str))

                # Generate a list of all n-grams of size n for the sentence
                n_grams_str = list(ngrams(og_sentence, 2))
                
                # Use the bigram collocation finder to get the best bigrams for the sentence
                finder_str = BigramCollocationFinder.from_words(og_sentence)
                best_bigrams_str = finder_str.nbest(bigram_measures.pmi, len(n_grams_str))

                # Filter out bigrams that contain common words from the current list of uncommon words
                best_uncommon_ngrams_str = [ngram for ngram in best_bigrams_str if (not any(p_ngrams in ngram for p_ngrams in common_words_str))]
                
                # Generate the final list of uncommon n-grams for string 1 by filtering the filtered bigrams and remaining uncommon words
                uncommon_ngrams_str = [''] * len(unc_str)
                count1 = len(unc_str)
                count2 = 0
                for b in best_uncommon_ngrams_str:
                    if b[0] in unc_str and b[1] in unc_str:
                        uncommon_ngrams_str[unc_str.index(b[0])] = " ".join(list(b))
                        count2 += 1
                        unc_str[unc_str.index(b[0])] = ''
                        unc_str[unc_str.index(b[1])] = ''
                        count1 -= 2
                    if count1 + count2 == lens[i]:
                        break
                if unc_str != [""] * len(unc_str):
                    for j in range(len(unc_str)):
                        if unc_str[j] != '':
                            uncommon_ngrams_str[j] = unc_str[j]
                uncommon_ngrams_str = remove_all(uncommon_ngrams_str, '')
                unc_str = uncommon_ngrams_str.copy()
            final_uncommon_str_i[nb_unc_str].append(unc_str)
        nb_unc_str += 1
    return final_uncommon_str_i

In [68]:
final_uncommon_str = ngram_distribution(uncommon_str_i, common_sentence)
final_uncommon_str

[[['name', 'is james', 'and i', 'work as', 'a data', 'scientists']],
 [['nail', 'fits', 'jane', 'andy', 'workable', 'scientist']]]

## 5) Apply the mask on the uncommon parts

In [69]:
def masking(common_sentence, uncommon_str):
    for i in range(len(uncommon_str)):
        mask = "[MASK] " * len(uncommon_str[i])
        common_sentence = common_sentence.replace("#", mask, 1)
    return " ".join(common_sentence.split())
masked_sentence = masking(common_sentence, final_uncommon_str[0])

In [70]:
for i in range(len(sentences)):
    print("sentence " + str(i + 1) +" : "+ sentences[i][0])
print("\nCommon parts of the 2 sentences with the uncommon parts masked :\n"+ masked_sentence) 
print("\nUncommon parts of each sentence : ")
for i in range(len(final_uncommon_str)):
    print("\nsentence " + str(i + 1) +" : "+str(final_uncommon_str[i]))

sentence 1 : hello my name is james and i work as a data scientists
sentence 2 : hello my nail fits jane andy workable scientist

Common parts of the 2 sentences with the uncommon parts masked :
hello my [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]

Uncommon parts of each sentence : 

sentence 1 : [['name', 'is james', 'and i', 'work as', 'a data', 'scientists']]

sentence 2 : [['nail', 'fits', 'jane', 'andy', 'workable', 'scientist']]
